In [1]:
import quandl
import datetime
from datetime import date
import pandas as pd
import numpy as np
quandl.ApiConfig.api_key = "63gdVnc_-LzW9XyB1Ajk"

List of potential stocks to invest in

In [2]:
stocksymlist = ["MSFT", "AXP", "BA", "CAT", "CVX", "CSCO", "KO", "DIS",  "XOM", "GE", "GS",
                "HD", "IBM", "JNJ", "JPM", "MCD", "MRK", "NKE", "PFE", "PG", "UTX", "UNH", "VZ",
                "V", "WMT"]

In [3]:
years_back = 1
tnow = date.today()
enddate = tnow.isoformat()
startyear = tnow.year - years_back
sd = tnow.replace(year=startyear)
#startmonth = tnow.month - 1
#startday = tnow.day + 1
#sd = tnow.replace(day=startday, month=startmonth)
startdate = sd.isoformat()
startdate

'2017-03-25'

Get the asset data from quandl

In [4]:
assets = []
for stocksym in stocksymlist:
    stocksymlo = stocksym.lower()
    stocksymup = stocksym.upper()
    stocksym = stocksymlo
    quandlstock = str("WIKI/")
    quandlstock = quandlstock + stocksymup
    stock = quandl.get(quandlstock, start_date=startdate, end_date=enddate)
    assets.append(stock)


Add continuous growth rates to stock info and calculate risk level (sigma) and return rate (drift).  Also, create a list of cgrs, which we'll use to calculate covariance

In [5]:
sigmas = []
drifts = []
cgrs = []
for stock in assets:
    price = np.array(stock.Close)
    pricep1 = np.roll(price,1)
    lnratio = price/pricep1
    cgr = np.log(lnratio)
    cgr[0]=999.99
    stock['cgr'] = cgr
    stock = stock[['Close','Volume','cgr']]
    
    sigma = np.std(cgr[1:])
    sigmas.append(sigma)
    drift = np.mean(cgr[1:])
    drifts.append(drift)
    cgrs.append(cgr[1:])

Put cgrs in numpy array and calculate covariances

In [6]:
cgrs = np.array(cgrs)
covmat = np.cov(cgrs)

Function to normalize weights:

In [7]:
def normalize_weights(weights):
    total = np.sum(weights)
    return weights/total

Function to calculate risk for a portfolio

In [8]:
def risk(weights):
    sig_sq = float(0)
    for i in range(len(weights)):
        for j in range(len(weights)):
            sig_sq += weights[i]*weights[j]*covmat[i][j]
    return sig_sq**0.5
            

Function to calculate return of portfolio

In [9]:
def return_rate(weights):
    rate = float(0)
    for i in range(len(weights)):
        rate += weights[i]*drifts[i]
    return rate

KL Divergence function.  For two normal distributions with std devs $\sigma_1$ and $\sigma_2$ and means $\mu_1$ and $\mu_2$, the KL Divergence is

$KL = \log\frac{\sigma_2}{\sigma_1} + \frac{\sigma_1^2 + (\mu_1 - \mu_2)^2}{2\sigma_2^2} - \frac{1}{2}$

In our function the first term will be called $a$ and the second term $b$

In [10]:
def KL(p_rate, p_sig, q_rate, q_sig):
    a = np.log(q_sig/p_sig)
    b = (p_sig**2 + (p_rate-q_rate)**2)/(2*q_sig**2)
    return a + b - 0.5
    

We want to find an upper limit on how good our sharpe ratio can be.  The best return we can have is simply the max of the returns of the individual assets.  The lowest risk is somewhat more difficult to find.  The risk is trickier to find, since the risk is lowered by diversification.  However, an evenly distributed portfolio is a good guess for the minimal risk, since that at least minimizes the $\pi_i \pi_j$ term in the summation.

In [11]:
target_rate = max(drifts)
safe_weights = np.array([1.0/len(assets) for i in range(len(assets))])*4
target_risk = risk(safe_weights)/4

Here is the formula we calculated for the partial of the divergence with respect to a weight

$\frac{\partial KL(p,q)}{\partial \pi_i} =  - \frac{\sum_{j=1}^n \pi_j \text{Cov} (R_i, R_j)}{2\sum_{i,j=1}^n \pi_i \pi_j \text{Cov} (R_i, R_j)} 
 + \frac{\sum_{j=1}^n \pi_j \text{Cov} (R_i, R_j)}{2\sigma_2^2} + \frac{r_j (\sum_{j=1}^n \pi_j r_j - \mu_2)}{\sigma_2^2}$

Now here are some functions to help find the gradient of the loss function

In [12]:
def weighted_cov_sum(i, weights):
    wsum = float(0)
    for j in range(len(weights)):
        if j==i:
            wsum += 2*weights[i]*covmat[i][i]
        else:
            wsum += weights[j]*covmat[i][j]
    return wsum

In [13]:
def gradient(weights):
    sig1 = risk(weights)
    mu1 = return_rate(weights)
    grad = np.zeros(len(weights))
    for i in range(len(grad)):
        wsum = weighted_cov_sum(i, weights)
        grad[i] = (wsum/(2*(target_risk**2))) + ((drifts[i]*mu1)/(target_risk**2)) - (wsum/(2*(sig1**2)))
    return grad

Here is the bit of code that does the gradient descent.

In [19]:
def optimize_sharpe(epochs, learn_rate):
    #initialize and normalize weights
    weights = np.random.normal(loc=0.5, scale=0.1, size=len(assets))
    weights = weights/np.sum(weights)
    last_error = float("inf")
    for epoch in range(epochs):
        #get a new set of samples
        xbatch = next_batch()
        
        #normalize the weights
        if np.min(weights)<0:
            weights = weights - np.min(weights)
        weights = weights/np.sum(weights)
        
        #take stddev and mean of predictions
        pred_sig = risk(weights)
        pred_rate = return_rate(weights)
        
        #compare to target using KL div
        error = KL(pred_rate, pred_sig, target_rate, target_risk)
        
        #get gradient for these weights
        grad = gradient(weights)
        
        #update weights
        weights += error*grad*learn_rate
        
        #change learning rate
        if error/last_error > 1.04:
            learn_rate *= 0.7
        elif error < last_error:
            learn_rate *= 1.05
        
        last_error = error
            
        if epoch%1000 == 0:
            print(error)
    
    if np.min(weights)<0:
        weights = weights-np.min(weights)
    weights = weights/np.sum(weights)
    #norm_weights = weights/np.sum(weights)

    return (weights, error)

(proportions, error) = optimize_sharpe(10000, 0.001)
(proportions, error)

0.0665924056286
0.0496012387644
0.0496012387644
0.0496012387644
0.0496012387644
0.0496012387644
0.0496012387644
0.0496012387644
0.0496012387644
0.0496012387644


(array([ 0.05309086,  0.05216694,  0.07933059,  0.07540222,  0.03583917,
         0.05614521,  0.01931915,  0.02769312,  0.02437967,  0.00270538,
         0.05588938,  0.04820473,  0.02596953,  0.02915392,  0.05396609,
         0.03437528,  0.01628248,  0.04795217,  0.03395631,  0.01372702,
         0.0397458 ,  0.04894376,  0.03356956,  0.04892027,  0.04327139]),
 0.049601238764434563)

Check that proportions add up to one.  Note that negative coefficients mean that you should be shorting that asset.  Since you gain capital from short selling a stock, it doesn't matter that the absolute values of all the assets exceeds 1.0.

In [20]:
np.sum(proportions)

1.0

Print out the optimal set of investments.

In [21]:
print("Portion of portfolio allocated to each asset:")
for i in range(len(assets)):
    print(stocksymlist[i] + ": ", proportions[i])
      
portfolio_risk = np.exp(risk(proportions))
portfolio_rate = np.exp(return_rate(proportions))
print("Daily risk: ", portfolio_risk, " standard deviation in daily return")
print("Daily return: ", portfolio_rate, " daily return")
print("Daily Sharpe = ", portfolio_rate/portfolio_risk)

yearly_risk = 100*np.exp((252**0.5)*risk(proportions))-100
yearly_return = 100*np.exp(252*return_rate(proportions))-100

print("Yearly risk: ", yearly_risk, " standard deviation in percent return")
print("Yearly return: ", yearly_return, " expected yearly percent return")

Portion of portfolio allocated to each asset:
MSFT:  0.0530908605688
AXP:  0.0521669411439
BA:  0.0793305866345
CAT:  0.0754022233442
CVX:  0.0358391733146
CSCO:  0.0561452095357
KO:  0.0193191515018
DIS:  0.0276931177897
XOM:  0.0243796745766
GE:  0.00270537708667
GS:  0.0558893789151
HD:  0.0482047347027
IBM:  0.0259695296038
JNJ:  0.029153920617
JPM:  0.0539660916195
MCD:  0.0343752788249
MRK:  0.0162824754119
NKE:  0.0479521664225
PFE:  0.0339563077635
PG:  0.0137270249858
UTX:  0.0397457950647
UNH:  0.0489437581202
VZ:  0.0335695633777
V:  0.048920265631
WMT:  0.0432713934432
Daily risk:  1.00701214877  standard deviation in daily return
Daily return:  1.00087023885  daily return
Daily Sharpe =  0.993900858167
Yearly risk:  11.7312165914  standard deviation in percent return
Yearly return:  24.5086274145  expected yearly percent return


While this may seem like an unusually high rate of return, the past year did see a boom in the stock market.  It is also a feasible value, as the fastest growing stock among those we considered grew at a rate of over 100 percent in the past year, as we can see below:

In [22]:
np.exp(target_rate*250)

2.0123361767928309

Also, it makes sense that the benefits of diversification are minimal since there was a booming market and therefore highly correlated stocks

Now we'll print out the risk and return for each of the assets we considered

In [24]:
print("Asset", "Yearly return", "Yearly risk")
best_sharpe = float(0)
for i in range(len(assets)):
    sharpe = np.exp(drifts[i])/np.exp(sigmas[i])
    if sharpe > best_sharpe:
        best_sharpe = sharpe
    print(stocksymlist[i], np.exp(drifts[i]), np.exp(sigmas[i]), sharpe)
print("Best sharpe of individual stock: ", best_sharpe)
print("Our Sharpe: ", portfolio_rate/portfolio_risk)

Asset Yearly return Yearly risk
MSFT 1.00152853019 1.01119876735 0.990436858239
AXP 1.00082632549 1.01058993132 0.990338706606
BA 1.00280110107 1.01383942912 0.989112350796
CAT 1.00194260151 1.01409893402 0.988012675988
CVX 1.00000399728 1.01123720115 0.988891623188
CSCO 1.00109627513 1.01209290113 0.989134766204
KO 1.00011737216 1.0070293243 0.993136295065
DIS 0.999707412881 1.01141320008 0.988426305685
XOM 0.999651547448 1.00911555271 0.990621485088
GE 0.996957337389 1.01546763452 0.981771652293
GS 1.00019039773 1.01368941688 0.986683278999
HD 1.00090181088 1.01072615221 0.990279917756
IBM 0.999405461088 1.01149223277 0.988050554134
JNJ 1.00019392648 1.00940911361 0.990870711387
JPM 1.00095132885 1.01092512801 0.990133988273
MCD 1.0008473959 1.00961043852 0.991320372412
MRK 0.999203574821 1.01098835987 0.988343302931
NKE 1.00069135708 1.01511032077 0.985795668316
PFE 1.00026843774 1.00925554652 0.991095309004
PG 0.999432383339 1.00787362727 0.991624700056
UTX 1.00074154002 1.01050891